In [1]:
import os
import argparse

import torch

import rlcard
from rlcard.utils import (
    get_device,
    set_seed,
    tournament,
    reorganize,
    Logger,
    plot_curve,
)
from rlcard.agents import (
    DQNAgent,
    RandomAgent,
    CDQNAgent
)

from rlcard.models.gin_rummy_rule_models import GinRummyNoviceRuleAgent


In [2]:
###Training DQN
device = get_device()
set_seed(42)
env = rlcard.make(
        'gin-rummy',
        config={
            'seed': 42,
        }
    )

agent = CDQNAgent(
                num_actions=env.num_actions,
                state_shape=env.state_shape[0],
                mlp_layers=[64,64],
                device=device,
                learning_rate = 0.00005,
                replay_memory_init_size = 100,
                number_discards = 3,
                execution_step = False,
                optimization_step = True,
            )

#path = 
#agent = CDQNAgent.from_checkpoint(checkpoint=torch.load(path))


--> Running on the GPU


"\nagent = CDQNAgent.from_checkpoint(checkpoint=torch.load('final/cdqn/cdqn_results_reward_shaped_5000/checkpoint_dqn.pt'))\n\n\nagent = DQNAgent(\n                num_actions=env.num_actions,\n                state_shape=env.state_shape[0],\n                discount_factor = 0.95,\n                mlp_layers=[128, 64, 32],\n                device=device,\n                epsilon_decay_steps=20000,\n                batch_size = 64,\n                learning_rate = 0.0001,\n                replay_memory_init_size = 100\n            )\n\n\n\ndiscount_factor = 0.99\nbatch_size = 32\nlearning_rate = 0.00005\nmlp_layers = [64,64]\nreplay_memory_init_size = 100\nreplay_memory_size = 20000\nagent = CDQNAgent(\n                num_actions=env.num_actions,\n                state_shape=env.state_shape[0],\n                mlp_layers=[64,64],\n                device=device,\n                learning_rate = 0.00005,\n                replay_memory_init_size = 100\n            )\n\nagent = DQNAgent(

In [3]:
###Training
from rlcard.models.gin_rummy_rule_models import GinRummyNoviceRuleAgent
agents = [agent, RandomAgent(num_actions = env.num_actions)]

env.set_agents(agents)


In [ ]:
###Training
with Logger('final/cdqnv2/cdqn_3_False') as logger:
    for episode in range(20000):
        trajectories, payoffs, _, _ = env.run(is_training = True)

        trajectories = reorganize(trajectories, payoffs)

        # Feed transitions into agent memory, and train the agent
        # Here, we assume that DQN always plays the first position
        # and the other players play randomly (if any)
        for ts in trajectories[0]:
            agents[0].feed(ts)
            #agents[1].feed(ts)
            
        # Evaluate the performance. Play with random agents.
        if episode % 100 == 0:
            logger.log_performance(
                episode,
                tournament(
                    env,
                    10000,
                )
            )
            
        agent.save_checkpoint('final/cdqnv2/cdqn_3_False')

    # Get the paths
    csv_path, fig_path = logger.csv_path, logger.fig_path


Moves:  117.415

----------------------------------------
  episode      |  0
  reward       |  ([-0.5142650000000005, -0.5430800000000006], [0.19934420644721712, 0.1963292594642933], [0.4995, 0.5005])
----------------------------------------
INFO - Step 100, rl-loss: 0.34437036514282227
INFO - Copied model parameters to target network.
INFO - Step 1100, rl-loss: 0.115605570375919344
INFO - Copied model parameters to target network.
INFO - Step 2100, rl-loss: 0.082787156105041596
INFO - Copied model parameters to target network.
INFO - Step 3100, rl-loss: 0.062596969306468966
INFO - Copied model parameters to target network.
INFO - Step 4100, rl-loss: 0.049029793590307236
INFO - Copied model parameters to target network.
INFO - Step 5100, rl-loss: 0.034597277641296394
INFO - Copied model parameters to target network.
INFO - Step 5616, rl-loss: 0.033472131937742235Moves:  113.1505

----------------------------------------
  episode      |  100
  reward       |  ([-0.5316749999999997, -0

In [ ]:
#plot_curve(csv_path, fig_path, 'cdqn')

# Save model
save_path = os.path.join('final/cdqn/cdqn_results_reward_shaped_6500', 'model.pth')
torch.save(agent, save_path)
print('Model saved in', save_path)

In [3]:
device = get_device()

# Seed numpy, torch, random
set_seed(42)

# Make the environment with seed
env = rlcard.make('gin-rummy', config={'seed': 42})

import torch

from rlcard.agents import RandomAgent
random = RandomAgent(num_actions=env.num_actions)

from rlcard.models.gin_rummy_rule_models import GinRummyNoviceRuleAgent
rule = GinRummyNoviceRuleAgent()

dqnRs = torch.load('final/dqn/dqn_results_reward_shaped_10000/model.pth')
dqnRs.set_device(device)

dqn = torch.load('experiments/gin_rummy_dqn_result_no_constraints/model.pth')
dqn.set_device(device)

rs1 = torch.load('final/rs/1/model.pth')
rs1.set_device(device)

rs2 = torch.load('final/rs/2/model.pth')
rs2.set_device(device)

rs3 = torch.load('final/rs/3/model.pth')
rs3.set_device(device)

rs4 = torch.load('final/rs/4/model.pth')
rs4.set_device(device)

rs5 = torch.load('final/rs/5/model.pth')
rs5.set_device(device)

cdqn3 = torch.load('final/cdqnv2/3/model.pth')
cdqn3.set_device(device)

env.set_agents([cdqn3, dqn])


--> Running on the GPU


In [4]:
for i in range(10):
    payoffs, unsafe, wins = tournament(env, 10000)
    print(payoffs)
    print(unsafe)
    print(wins)
    print("="*30) 

Moves:  119.9645
[-0.5542500000000002, -0.5499600000000013]
[0.19269867335753493, 0.19309045592654495]
[0.4855, 0.5145]


KeyboardInterrupt: 

^C


In [1]:
def read_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    episodes = list(range(0, 9901, 100))
    rewards = []
    for line in lines:
        if 'reward' in line:
            parts = line.split('|')
            reward_list = eval(parts[1])
            
            rewards.append(reward_list[0][0])

    return episodes, rewards


import matplotlib.pyplot as plt
file_paths = [
    'final/rs/1/log.txt',
    'final/rs/2/log.txt',
    'final/rs/3/log.txt',
    'final/rs/4/log.txt',
    'final/rs/5/log.txt',
    'final/dqn/dqn_results_reward_shaped/results_10k.txt',
]

fig, ax = plt.subplots()

for i, file_path in enumerate(file_paths, start=1):
    episodes, rewards = read_data(file_path)
    
    label = f'cdqn-{i}'
    print(i)
    ax.plot(episodes, rewards, label=label)

ax.set(xlabel='episode', ylabel='reward')
ax.legend()
ax.grid()

plt.show()

fig.savefig('final/testfig')

In [2]:
import numpy as np

from collections import OrderedDict

dqn = torch.load('experiments/gin_rummy_dqn_result_no_constraints/model.pth')
dqn.set_device(device)

dqnRs = torch.load('final/dqn/dqn_results_reward_shaped_10000/model.pth')
dqnRs.set_device(device)

cdqn3 = torch.load('final/cdqnv2/3/model.pth')
cdqn3.set_device(device)

state = {'obs': np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0]]), 'legal_actions': OrderedDict([(6, None), (18, None), (20, None), (25, None), (26, None), (41, None), (44, None), (47, None), (48, None), (56, None), (57, None)]), 'raw_legal_actions': [6, 18, 20, 25, 26, 41, 44, 47, 48, 56, 57], 'raw_obs': np.array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0]])}

cdqn3.eval_step(state)

NameError: name 'torch' is not defined